# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import getpass  # to get the password without showing the input
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

········


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [3]:
engine.execute('''
DROP TABLE IF EXISTS rentals_may;
''')

In [4]:
engine.execute('''
CREATE TABLE rentals_may (
rental_id int NOT NULL AUTO_INCREMENT,
rental_date datetime NOT NULL,
inventory_id mediumint unsigned NOT NULL,
customer_id smallint unsigned NOT NULL,
return_date datetime DEFAULT NULL,
staff_id tinyint unsigned NOT NULL,
last_update timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
PRIMARY KEY (`rental_id`)
);
''')

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [5]:
engine.execute('''
INSERT INTO rentals_may (rental_id, rental_date, inventory_id, customer_id, return_date, staff_id, last_update)
SELECT *
FROM rental
WHERE month(rental_date) = 5;
''')

## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [6]:
engine.execute('''
DROP TABLE IF EXISTS rentals_june;
''')

In [7]:
engine.execute(
'''
CREATE TABLE rentals_june (
rental_id int NOT NULL AUTO_INCREMENT,
rental_date datetime NOT NULL,
inventory_id mediumint unsigned NOT NULL,
customer_id smallint unsigned NOT NULL,
return_date datetime DEFAULT NULL,
staff_id tinyint unsigned NOT NULL,
last_update timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
PRIMARY KEY (`rental_id`)
);
''')


## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [8]:
engine.execute('''
INSERT INTO rentals_june (rental_id, rental_date, inventory_id, customer_id, return_date, staff_id, last_update)
SELECT *
FROM rental
WHERE month(rental_date) = 6;
''')

## Check the number of rentals for each customer for May.


In [9]:
engine.execute('''
SELECT distinct count(customer_id)
FROM rentals_may;
''')

## Check the number of rentals for each customer for June.
 

In [10]:
engine.execute('''
SELECT distinct count(customer_id)
FROM rentals_june;
''')

## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [11]:
query = '''
        SELECT * FROM rentals_may;
        '''

rentals_may = pd.read_sql_query(query, engine)
rentals_may

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


In [12]:
query = '''
        SELECT * FROM rentals_june;
        '''

rentals_june = pd.read_sql_query(query, engine)
rentals_june

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
2307,3466,2005-06-21 22:13:33,371,226,2005-06-25 21:01:33,2,2006-02-15 21:30:53
2308,3467,2005-06-21 22:19:25,729,543,2005-06-27 00:03:25,2,2006-02-15 21:30:53
2309,3468,2005-06-21 22:43:45,2899,100,2005-06-30 01:49:45,1,2006-02-15 21:30:53
2310,3469,2005-06-21 22:48:59,4087,181,2005-06-28 19:32:59,1,2006-02-15 21:30:53


## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [13]:
# First part: creating a join between the two dataframes created before
merged = rentals_may.merge(rentals_june, how='outer', on='customer_id')
merged

,rental_id_x,rental_date_x,inventory_id_x,customer_id,return_date_x,staff_id_x,last_update_x,rental_id_y,rental_date_y,inventory_id_y,return_date_y,staff_id_y,last_update_y
0,1.0,2005-05-24 22:53:30,367.0,130,2005-05-26 22:04:30,1.0,2006-02-15 21:30:53,1630.0,2005-06-16 07:55:01,2413.0,2005-06-19 06:38:01,1.0,2006-02-15 21:30:53
1,1.0,2005-05-24 22:53:30,367.0,130,2005-05-26 22:04:30,1.0,2006-02-15 21:30:53,1864.0,2005-06-17 01:39:47,1815.0,2005-06-24 19:39:47,2.0,2006-02-15 21:30:53
2,1.0,2005-05-24 22:53:30,367.0,130,2005-05-26 22:04:30,1.0,2006-02-15 21:30:53,2163.0,2005-06-17 23:46:16,2600.0,2005-06-22 22:48:16,2.0,2006-02-15 21:30:53
3,1.0,2005-05-24 22:53:30,367.0,130,2005-05-26 22:04:30,1.0,2006-02-15 21:30:53,2292.0,2005-06-18 07:37:48,173.0,2005-06-20 02:45:48,2.0,2006-02-15 21:30:53
4,1.0,2005-05-24 22:53:30,367.0,130,2005-05-26 22:04:30,1.0,2006-02-15 21:30:53,2535.0,2005-06-19 01:39:04,901.0,2005-06-28 01:33:04,2.0,2006-02-15 21:30:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4707,NaN,NaT,NaN,487,NaT,NaN,NaT,3100.0,2005-06-20 16:47:57,2428.0,2005-06-26 16:59:57,1.0,2006-02-15 21:30:53
4708,NaN,NaT,NaN,555,NaT,NaN,NaT,3232.0,2005-06-21 02:30:37,1972.0,2005-06-29 03:10:37,1.0,2006-02-15 21:30:53
4709,NaN,NaT,NaN,335,NaT,NaN,NaT,3329.0,2005-06-21 09:20:31,3506.0,2005-06-22 10:00:31,2.0,2006-02-15 21:30:53
4710,NaN,NaT,NaN,226,NaT,NaN,NaT,3414.0,2005-06-21 16:58:50,2929.0,2005-06-24 17:26:50,1.0,2006-02-15 21:30:53


In [14]:
len(merged['customer_id'].unique())

598

In [15]:
merged.columns

Index(['rental_id_x', 'rental_date_x', 'inventory_id_x', 'customer_id',
       'return_date_x', 'staff_id_x', 'last_update_x', 'rental_id_y',
       'rental_date_y', 'inventory_id_y', 'return_date_y', 'staff_id_y',
       'last_update_y'],
      dtype='object')

In [16]:
merged['customer_id'].value_counts()

197    64
371    42
176    40
506    35
256    35
       ..
580     1
419     1
335     1
440     1
2       1
Name: customer_id, Length: 598, dtype: int64

In [47]:
merged.dtypes

rental_id_x              float64
rental_date_x     datetime64[ns]
inventory_id_x           float64
customer_id                int64
return_date_x     datetime64[ns]
staff_id_x               float64
last_update_x     datetime64[ns]
rental_id_y              float64
rental_date_y     datetime64[ns]
inventory_id_y           float64
return_date_y     datetime64[ns]
staff_id_y               float64
last_update_y     datetime64[ns]
dtype: object

In [95]:
# Here I was wanting to create a dataframe with 'customer_id' and other 2 columns with the rents from may and june
# But I was not able to group the values counting the number of occurences each month (next coding cell)
# I did it, but when I was checking the output it's wrong.
# For example, customer_id = 1 had 2 rentals on may and 7 on june, and not 14 each month

d = {'customer_id': pd.Series(merged['customer_id']), 'rentals_may': pd.Series(merged['rental_date_x'].dt.month)
    , 'rentals_june': pd.Series(merged['rental_date_y'].dt.month)}

df = pd.DataFrame(data=d)
df

,customer_id,rentals_may,rentals_june
0,130,5.0,6.0
1,130,5.0,6.0
2,130,5.0,6.0
3,130,5.0,6.0
4,130,5.0,6.0
...,...,...,...
4707,487,NaN,6.0
4708,555,NaN,6.0
4709,335,NaN,6.0
4710,226,NaN,6.0


In [98]:
rent_by_client = df.groupby('customer_id')['rentals_may', 'rentals_june'].count()
rent_by_client

<ipython-input-98-f43cd5eaad82>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rent_by_client = df.groupby('customer_id')['rentals_may', 'rentals_june'].count()


,rentals_may,rentals_june
customer_id,,
1,14,14
2,1,1
3,8,8
4,0,6
5,15,15
...,...,...
595,2,2
596,12,12
597,6,6


In [99]:
# Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
def check_rental(may, june):
        if may > june:
            return print('More rentals in May')
        if june > may:
            return print('More rentals in June')
        else:
            return print('Same rentals in both months')

In [100]:
# Applying the check
rent_by_client['check'] = rent_by_client.apply(lambda row: check_rental(row['rentals_may'], row['rentals_june']), axis = 1)

Same rentals in both months
Same rentals in both months
Same rentals in both months
More rentals in June
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
More rentals in June
Same rentals in both months
More rentals in June
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
Same rentals in both months
More rentals in June
More rentals in June
Same rentals in both months
Same rentals in both months
More rentals in June
Same rentals in both months
Same rentals in both months
Same rentals in both months
More r